In [4]:
# 1st Example: Integer hyperparameters and float hyperparameters
from ConfigSpace import ConfigurationSpace

cs = ConfigurationSpace(
    space={
        "C": (-1.0, 1.0),  # Note the decimal to make it a float
        "max_iter": (10, 100),
    },
    seed=1234,
)

In [ ]:
# 2nd Example: Categorical hyperparameters and conditions#
from ConfigSpace import ConfigurationSpace, Categorical, Float, Integer

kernel_type = Categorical('kernel_type', ['linear', 'poly', 'rbf', 'sigmoid'])
degree = Integer('degree', bounds=(2, 4), default=2)
coef0 = Float('coef0', bounds=(0, 1), default=0.0)
gamma = Float('gamma', bounds=(1e-5, 1e2), default=1, log=True)

cs = ConfigurationSpace()
cs.add([kernel_type, degree, coef0, gamma])

from ConfigSpace import EqualsCondition, InCondition, OrConjunction

# read as: "degree is active if kernel_type == 'poly'"
cond_1 = EqualsCondition(degree, kernel_type, 'poly')

# read as: "coef0 is active if (kernel_type == 'poly' or kernel_type == 'sigmoid')"
# You could also define this using an InCondition as shown below
cond_2 = OrConjunction(
    EqualsCondition(coef0, kernel_type, 'poly'),
    EqualsCondition(coef0, kernel_type, 'sigmoid')
)

# read as: "gamma is active if kernel_type in ['rbf', 'poly', 'sigmoid']"
cond_3 = InCondition(gamma, kernel_type, ['rbf', 'poly','sigmoid'])



Configuration space object:
  Hyperparameters:
    coef0, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0
    degree, Type: UniformInteger, Range: [2, 4], Default: 2
    gamma, Type: UniformFloat, Range: [1e-05, 100.0], Default: 1.0, on log-scale
    kernel_type, Type: Categorical, Choices: {linear, poly, rbf, sigmoid}, Default: linear



In [ ]:
# 3rd Example: Forbidden clauses#
from ConfigSpace import ConfigurationSpace, Categorical, Constant

cs = ConfigurationSpace()

penalty = Categorical("penalty", ["l1", "l2"], default="l2")
loss = Categorical("loss", ["hinge", "squared_hinge"], default="squared_hinge")
dual = Constant("dual", "False")
cs.add([penalty, loss, dual])

from ConfigSpace import ForbiddenEqualsClause, ForbiddenAndConjunction

penalty_and_loss = ForbiddenAndConjunction(
    ForbiddenEqualsClause(penalty, "l1"),
    ForbiddenEqualsClause(loss, "hinge")
)
constant_penalty_and_loss = ForbiddenAndConjunction(
    ForbiddenEqualsClause(dual, "False"),
    ForbiddenEqualsClause(penalty, "l2"),
    ForbiddenEqualsClause(loss, "hinge")
)
penalty_and_dual = ForbiddenAndConjunction(
    ForbiddenEqualsClause(dual, "False"),
    ForbiddenEqualsClause(penalty, "l1")
)

In [ ]:
# 4th Example Serialization#
from pathlib import Path
from ConfigSpace import ConfigurationSpace

path = Path("configspace.yaml")
cs = ConfigurationSpace(
    space={
        "C": (-1.0, 1.0),  # Note the decimal to make it a float
        "max_iter": (10, 100),
    },
    seed=1234,
)
cs.to_yaml(path)
loaded_cs = ConfigurationSpace.from_yaml(path)

with path.open() as f:
    print(f.read())

In [3]:
# 5th Example: Placing priors on the hyperparameters#
import numpy as np
from ConfigSpace import ConfigurationSpace, Float, Categorical, Beta, Normal

cs = ConfigurationSpace(
    space={
        "lr": Float(
            'lr',
            bounds=(1e-5, 1e-1),
            default=1e-3,
            log=True,
            distribution=Normal(1e-3, 1e-1)
        ),
        "dropout": Float(
            'dropout',
            bounds=(0, 0.99),
            default=0.25,
            distribution=Beta(alpha=2, beta=4)
        ),
        "activation": Categorical(
            'activation',
            items=['tanh', 'relu'],
            weights=[0.2, 0.8]
        ),
    },
    seed=1234,
)
print(cs)


Configuration space object:
  Hyperparameters:
    activation, Type: Categorical, Choices: {tanh, relu}, Default: relu, Probabilities: [0.2 0.8]
    dropout, Type: BetaFloat, Alpha: 2.0, Beta: 4.0, Range: [0.0, 0.99], Default: 0.25
    lr, Type: NormalFloat, Mu: 0.001, Sigma: 0.1, Range: [1e-05, 0.1], Default: 0.001, on log-scale



In [ ]:
from ConfigSpace import Configuration


In [4]:
from smac import Scenario

def generate_scenario(cs):
    scenario = Scenario({
        'run_obj': 'quality',
        'output_dir': './automl_results',
        'shared_model': False,
        'multi_objectives': ['validation_loss'],
        'overall_obj': 'validation_loss',
        'deterministic': False,
        'wallclock_limit': 3600.0,
        'abort_on_first_run_crash': True,
        'limit_resources': True,
        'memory_limit': 16000.0,
        'cutoff': 300.0
    })
    return scenario

scenario = generate_scenario(cs)

In [6]:
import numpy as np
from types import SimpleNamespace
X = np.random.rand(10, 5)
y = np.random.rand(10, 1)


mydict = dict(X=X, y=y)


mydict = SimpleNamespace(**mydict)
mydict.X

array([[0.98525351, 0.781331  , 0.90316584, 0.42289328, 0.0836624 ],
       [0.09098289, 0.45530062, 0.8913916 , 0.55417661, 0.19638563],
       [0.34897899, 0.89023083, 0.46529547, 0.62919728, 0.78103712],
       [0.21140623, 0.78827005, 0.41219888, 0.18284856, 0.95699989],
       [0.4160139 , 0.39566742, 0.68413273, 0.9207471 , 0.81198847],
       [0.1554572 , 0.28036314, 0.04413612, 0.39658158, 0.05237825],
       [0.56122639, 0.55022277, 0.39825877, 0.58396533, 0.46107929],
       [0.69087577, 0.2232384 , 0.04952998, 0.64866486, 0.50059245],
       [0.69802054, 0.18925913, 0.25659784, 0.16636338, 0.98976883],
       [0.40259054, 0.33274224, 0.01593082, 0.50959103, 0.43491217]])

In [10]:
from keras.datasets import mnist

ModuleNotFoundError: No module named 'keras'

In [3]:
# load mnist dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target


In [42]:
from ConfigSpace import ConfigurationSpace, Categorical, Float, Integer, ForbiddenAndConjunction, ForbiddenEqualsClause, EqualsCondition

def get_configspace():
    cs = ConfigurationSpace(seed=1234)

    learning_rate = Categorical('learning_rate', ['constant', 'invscaling', 'adaptive'])
    eta0 = Float('eta0', bounds=(0.0001, 1.0), default=0.01, log=True)
    max_iter = Integer('max_iter', bounds=(100, 1000), default=200)
    tol = Float('tol', bounds=(1e-6, 1e-2), default=1e-4, log=True)
    early_stopping = Categorical('early_stopping', ['True', 'False'], default='False')
    validation_fraction = Float('validation_fraction', bounds=(0.01, 0.5), default=0.1)
    n_jobs = Integer('n_jobs', bounds=(1, 10), default=1)
    random_state = Integer('random_state', bounds=(0, 100), default=42)

    cs.add([learning_rate, eta0, max_iter, tol, early_stopping, validation_fraction, n_jobs, random_state])

    # cond_eta0 = EqualsCondition(eta0, learning_rate, 'constant')
    # cs.add(cond_eta0)

    # forbidden_eta0_and_max_iter = ForbiddenAndConjunction(
    #     ForbiddenEqualsClause(eta0, 0.01),
    #     ForbiddenEqualsClause(max_iter, 100)
    # )
    # cs.add(forbidden_eta0_and_max_iter)

    return cs

cs = get_configspace()
print(cs)

Configuration space object:
  Hyperparameters:
    early_stopping, Type: Categorical, Choices: {True, False}, Default: False
    eta0, Type: UniformFloat, Range: [0.0001, 1.0], Default: 0.01, on log-scale
    learning_rate, Type: Categorical, Choices: {constant, invscaling, adaptive}, Default: constant
    max_iter, Type: UniformInteger, Range: [100, 1000], Default: 200
    n_jobs, Type: UniformInteger, Range: [1, 10], Default: 1
    random_state, Type: UniformInteger, Range: [0, 100], Default: 42
    tol, Type: UniformFloat, Range: [1e-06, 0.01], Default: 0.0001, on log-scale
    validation_fraction, Type: UniformFloat, Range: [0.01, 0.5], Default: 0.1



In [44]:
from smac.scenario import Scenario

def generate_scenario(cs):
    scenario = Scenario(
        configspace=cs,
        deterministic=True,
        n_trials=10,
    )
    return scenario
scenario = generate_scenario(cs)
print(scenario)

Scenario(configspace=Configuration space object:
  Hyperparameters:
    early_stopping, Type: Categorical, Choices: {True, False}, Default: False
    eta0, Type: UniformFloat, Range: [0.0001, 1.0], Default: 0.01, on log-scale
    learning_rate, Type: Categorical, Choices: {constant, invscaling, adaptive}, Default: constant
    max_iter, Type: UniformInteger, Range: [100, 1000], Default: 200
    n_jobs, Type: UniformInteger, Range: [1, 10], Default: 1
    random_state, Type: UniformInteger, Range: [0, 100], Default: 42
    tol, Type: UniformFloat, Range: [1e-06, 0.01], Default: 0.0001, on log-scale
    validation_fraction, Type: UniformFloat, Range: [0.01, 0.5], Default: 0.1
, name=None, output_directory=PosixPath('smac3_output'), deterministic=True, objectives='cost', crash_cost=inf, termination_cost_threshold=inf, walltime_limit=inf, cputime_limit=inf, trial_walltime_limit=None, trial_memory_limit=None, n_trials=10, use_default_config=False, instances=None, instance_features=None, min

In [32]:
print([p.name for p in list(cs.values())])

['early_stopping', 'eta0', 'learning_rate', 'max_iter', 'n_jobs', 'random_state', 'tol', 'validation_fraction']


In [45]:
scenario.configspace

Configuration space object:
  Hyperparameters:
    early_stopping, Type: Categorical, Choices: {True, False}, Default: False
    eta0, Type: UniformFloat, Range: [0.0001, 1.0], Default: 0.01, on log-scale
    learning_rate, Type: Categorical, Choices: {constant, invscaling, adaptive}, Default: constant
    max_iter, Type: UniformInteger, Range: [100, 1000], Default: 200
    n_jobs, Type: UniformInteger, Range: [1, 10], Default: 1
    random_state, Type: UniformInteger, Range: [0, 100], Default: 42
    tol, Type: UniformFloat, Range: [1e-06, 0.01], Default: 0.0001, on log-scale
    validation_fraction, Type: UniformFloat, Range: [0.01, 0.5], Default: 0.1

In [46]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from ConfigSpace import Configuration

def train(cfg: Configuration, seed: int, dataset: dict) -> float:
    """
    Train a neural network model on the given dataset.

    Args:
    - cfg (Configuration): A Configuration object containing hyperparameters.
    - seed (int): The random seed for reproducibility.
    - dataset (dict): A dictionary containing the feature matrix 'X' and label vector 'y'.

    Returns:
    - loss (float): The average training loss over 10 epochs.
    """

    # Get input and output dimensions dynamically from the dataset
    input_size = dataset['X'].shape[1]
    num_classes = len(np.unique(dataset['y']))

    # Get hyperparameters from the configuration
    learning_rate = cfg.get('learning_rate')
    eta0 = cfg.get('eta0')
    max_iter = cfg.get('max_iter')
    tol = cfg.get('tol')
    early_stopping = cfg.get('early_stopping') == 'True'
    validation_fraction = cfg.get('validation_fraction')
    n_jobs = cfg.get('n_jobs')
    warm_start = cfg.get('warm_start') == 'True'
    momentum = cfg.get('momentum')
    nesterovs_momentum = cfg.get('nesterovs_momentum') == 'True'
    power_t = cfg.get('power_t')

    # Create a neural network model
    if learning_rate == 'constant':
        # Use SGD with lr=eta0
        model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=max_iter, tol=tol, 
                               early_stopping=early_stopping, validation_fraction=validation_fraction, 
                               random_state=seed, learning_rate_init=eta0, momentum=momentum, 
                               nesterovs_momentum=nesterovs_momentum)
    elif learning_rate == 'invscaling':
        # Use SGD with lr=eta0 and momentum=power_t
        model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=max_iter, tol=tol, 
                               early_stopping=early_stopping, validation_fraction=validation_fraction, 
                               random_state=seed, learning_rate_init=eta0, momentum=momentum, 
                               nesterovs_momentum=nesterovs_momentum, power_t=power_t)
    elif learning_rate == 'adaptive':
        # Use Adam or equivalent with lr=eta0
        model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=max_iter, tol=tol, 
                               early_stopping=early_stopping, validation_fraction=validation_fraction, 
                               random_state=seed, learning_rate_init=eta0, solver='adam')

    # Train the model
    model.fit(dataset['X'], dataset['y'])

    # Calculate the average training loss over 10 epochs
    losses = []
    for _ in range(10):
        model.fit(dataset['X'], dataset['y'])
        losses.append(log_loss(dataset['y'], model.predict_proba(dataset['X'])))
    loss = np.mean(losses)

    return loss


In [47]:
cs = get_configspace()
print(type(cs))  # Should output: <class 'ConfigSpace.configuration_space.ConfigurationSpace'>


<class 'ConfigSpace.configuration_space.ConfigurationSpace'>


In [ ]:
from smac import HyperparameterOptimizationFacade, Scenario

smac = HyperparameterOptimizationFacade(
            scenario,
            train,  # We pass the target function here
            overwrite=True,  # Overrides any previous results that are found that are inconsistent with the meta-data
        )
smac.optimize()


[INFO][abstract_initial_design.py:87] Reducing the number of initial configurations from 80 to 2 (max_ratio == 0.25).
[WARNING][target_function_runner.py:74] The argument dataset is not set by SMAC: Consider removing it from the target function.
[INFO][abstract_initial_design.py:139] Using 2 initial design configurations and 0 additional configurations.
[INFO][abstract_intensifier.py:307] Using only one seed for deterministic scenario.
[WARNING][abstract_runner.py:135] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[WARNING][abstract_runner.py:141] Traceback: Traceback (most recent call last):
  File "/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/runner/target_function_runner.py", line 190, in run
    rval = self(config_copy, target_function, kwargs)
  File "/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/runner/target_function_runner.py", line 264, in __call__
    return algorithm(config, **algorithm_kw

/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_ar

[WARNING][abstract_runner.py:135] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[WARNING][abstract_runner.py:141] Traceback: Traceback (most recent call last):
  File "/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/runner/target_function_runner.py", line 190, in run
    rval = self(config_copy, target_function, kwargs)
  File "/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/runner/target_function_runner.py", line 264, in __call__
    return algorithm(config, **algorithm_kwargs)
TypeError: train() missing 1 required positional argument: 'dataset'


[WARNING][abstract_runner.py:135] Target function returned infinity or nothing at all. Result is treated as CRASHED and cost is set to inf.
[WARNING][abstract_runner.py:141] Traceback: Traceback (most recent call last):
  File "/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/runner/target_function_runner.py", line 190, in run
    rval = self(config_c

/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest.py:222: RuntimeWarning: Mean of empty slice
  preds_as_array = np.log(np.nanmean(np.exp(preds_as_array), axis=2) + VERY_SMALL_NUMBER)
/opt/anaconda3/envs/SMAC/lib/python3.10/site-packages/smac/model/random_forest/random_forest

AttributeError: 'HyperparameterOptimizationFacade' object has no attribute 'run'

In [41]:
configspace = ConfigurationSpace({"C": (0.100, 1000.0)})
print(configspace)

Configuration space object:
  Hyperparameters:
    C, Type: UniformFloat, Range: [0.1, 1000.0], Default: 500.05



In [27]:
from ConfigSpace import Configuration, ConfigurationSpace

import numpy as np
from smac import HyperparameterOptimizationFacade, Scenario
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

iris = datasets.load_iris()


def train(config: Configuration, seed: int = 0) -> float:
    classifier = SVC(C=config["C"], random_state=seed)
    scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
    return 1 - np.mean(scores)


configspace = ConfigurationSpace({"C": (0.100, 1000.0)})

# Scenario object specifying the optimization environment
scenario = Scenario(configspace, deterministic=True, n_trials=200)

# Use SMAC to find the best configuration/hyperparameters
smac = HyperparameterOptimizationFacade(scenario, train)
incumbent = smac.optimize()

[INFO][abstract_initial_design.py:139] Using 10 initial design configurations and 0 additional configurations.
[INFO][smbo.py:509] Continuing from previous run.
[INFO][smbo.py:278] Optimization process was already finished. Returning incumbent...
